#  Run Pix2Pix
Run this notebook for generating pictures in the paper. Make sure you use 10x dataset. <br>
This notebook is optimized for using pytorch (look at the environment on the top right). <br>
This is the main folder path: [~/ml/](http://localhost:8888/tree/ml)<br>
Image dataset is located here: [~/ml/dataset_oct_histology/](http://localhost:8888/tree/ml/dataset_oct_histology)<br>
[Github Link](https://github.com/junyanz/pytorch-CycleGAN-and-pix2pix)<br>
<br>
## Install

In [ ]:
# Set up general varibles
root_path = '~/ml/'
dataset_path = root_path + 'dataset_oct_histology/'
code_main_folder = root_path + 'pix2pix_and_CycleGAN/'

# Install environment dependencies
!pip install --upgrade pip
!pip install opencv-python

# Clear code, start a fresh copy
import shutil
import os
if os.path.isdir(os.path.expanduser(code_main_folder)):
    shutil.rmtree(os.path.expanduser(code_main_folder))
    
# Get main model
!git clone --single-branch https://github.com/junyanz/pytorch-CycleGAN-and-pix2pix {code_main_folder}
!pip install -r {code_main_folder}requirements.txt

## Build Dataset
This library requires OCT and histology images to be paired together.<br>
Code below merges images from  [patches_1024px_512px](http://localhost:8888/tree/ml/dataset_oct_histology/patches_1024px_512px) to [patches_1024px_512px_combined](http://localhost:8888/tree/ml/dataset_oct_histology/patches_1024px_512px_combined)

In [ ]:
import image_combine_aux_functions

# Define train & test patches folders
train_patches_folder = dataset_path + 'patches_1024px_512px/'
train_patches_folder_combined = train_patches_folder[:-1] + '_combined/'
test_patches_folder = dataset_path + 'patches_1024px_512px/'
test_patches_folder_combined = test_patches_folder[:-1] + '_combined/'

# Generate combine train dataset of A and B
image_combine_aux_functions.combine_images(
    img_fold_A = (train_patches_folder + 'train_A/'),
    img_fold_B = (train_patches_folder + 'train_B/'),
    img_fold_AB = (train_patches_folder_combined + 'train/'))

# Make sure that in train_patches_folder_combined there is no test folder.
# This prevents mistake leakage of data.
image_combine_aux_functions.clear_dir(train_patches_folder_combined + 'test/')

## Train
Run code below to train model.<br>
Results can be viewed here: [~/ml/checkpoints/pix2pix/web/index.html](http://localhost:8888/view/ml/checkpoints/pix2pix/web/index.html) as the model trains.<br>

In [ ]:
# Default setting includes flip which trains on left-right flips as well.
#'--preprocess crop' allows user to load larger than 256X256 images and just randomly crop the right size for training.
#  this is not as recommended because most of the crops will be just black with no information
# If model is stuck, restart using --continue_train --epoch_count <number> to get numbering right.
!python {code_main_folder}train.py --name pix2pix --dataroot {train_patches_folder_combined} --model pix2pix --checkpoints_dir {root_path}checkpoints

## Test

Main test results can be viewed here: [~/ml/results/pix2pix/test_latest/index.html](http://localhost:8888/view/ml/results/pix2pix/test_latest/index.html) after test command

Latent space values are visible here: [~/ml/results/pix2pix/feats/center/](http://localhost:8888/tree/ml/results/pix2pix/feats/center/)


In [ ]:
# Generate combine train dataset of A and B
image_combine_aux_functions.combine_images(
    img_fold_A = (test_patches_folder + 'test_A/'),
    img_fold_B = (test_patches_folder + 'test_B/'),
    img_fold_AB = (test_patches_folder_combined + 'test/'))

# Copy code patch to generate latent space
shutil.copy('networks.py',os.path.expanduser(code_main_folder + "models/"))
shutil.copy('pix2pix_model.py',os.path.expanduser(code_main_folder + "models/"))

# Main test results + weights at the center
#  --preprocess none allows to directly process a non 256x256 images using the convolutional proprety of the network
!python {code_main_folder}test.py --name pix2pix --dataroot {test_patches_folder_combined} --model pix2pix --checkpoints_dir {root_path}checkpoints --results_dir {root_path}results --num_test 10000

# Generate images and features on the train dataset as well
!python {code_main_folder}test.py --name pix2pix --dataroot {test_patches_folder_combined} --model pix2pix --checkpoints_dir {root_path}checkpoints --results_dir {root_path}results --num_test 10000 --phase train